In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_cola = pd.read_excel('/content/CocaCola_Sales_Rawdata.xlsx')
df_cola.head()

In [ ]:
df_cola.info()

In [ ]:
df_cola.isnull().sum()

In [ ]:
df_cola.describe()

In [ ]:
df_cola.shape

In [ ]:

# Converting quarterly periods to datetime in pandas
df_cola['Quarter_Year'] = df_cola['Quarter'].str.split('_').apply(lambda x: ' 19'.join(x[:]))
df_cola.head(10)

In [ ]:

# Converting into datetime formate as the index was not in correct format
df_cola['Date'] = (pd.to_datetime(df_cola['Quarter_Year'].str.split(' ').apply(lambda x: ''.join(x[::-1])),dayfirst=True))
df_cola.head(10)

In [ ]:
df_cola['Quarters'] = 0

for i in range(42):
    n = df_cola['Quarter'][i]
    df_cola['Quarters'][i] = n[0:2]

df_cola.head(10)

In [ ]:
dummy = pd.DataFrame(pd.get_dummies(df_cola['Quarters']))
cola1 = pd.concat((df_cola, dummy), axis=1)

cola1.head()

In [ ]:
cola1.drop(['Quarter','Quarter_Year'] , axis =1, inplace = True)
cola1.head()

In [ ]:
cola1.set_index('Date',inplace=True)
cola1.index.year

In [ ]:
cola1.head()

In [ ]:
df_cola.head()

In [ ]:
df_cola['Date'] = pd.to_datetime(df_cola['Date'], format="%b-%y")

In [ ]:
df_cola['Year'] = df_cola['Date'].dt.year

In [ ]:
df_cola["month"] = df_cola.Date.dt.strftime("%b")

In [ ]:
plt.figure(figsize = (14,8))
Pivot_data = pd.pivot_table(data = df_cola,values= 'Sales',index = 'Year', columns= 'month',aggfunc = 'mean')
sns.heatmap(data = Pivot_data,cmap = 'plasma',annot = True,fmt='g')

In [ ]:
# Sales of Coca-Cola

plt.figure(figsize=(12,5))
plt.plot(df_cola['Sales'], color = 'green',marker= '*')
plt.show()

In [ ]:
# Year wise Sales

plt.figure(figsize = (14,6))
plt.xlabel('Year')
plt.ylabel('Sales')
sns.lineplot(data=df_cola,x = df_cola['Year'],y = df_cola['Sales'],color='orange')

In [ ]:
# Quarter wise Sales

plt.figure(figsize = (14,6))
plt.xlabel('Quarters')
plt.ylabel('Sales')
sns.lineplot(data=df_cola,x = df_cola['Quarters'],y = df_cola['Sales'],color='orange')

In [ ]:
#Checking the outliers

plt.figure(figsize = (14,6))
plt.xlabel('Year')
plt.ylabel('Sales')
sns.boxplot(data = df_cola,x = df_cola['Year'],y = df_cola['Sales'])
plt.show()

In [ ]:
plt.figure(figsize = (14,6))
plt.xlabel('Quarters')
plt.ylabel('Sales')
sns.boxplot(data = df_cola,x = df_cola['Quarters'],y = df_cola['Sales'])
plt.show()

In [ ]:
# Distribution of the Sales column

plt.figure(figsize = (10,8))
sns.distplot(df_cola['Sales'],color = 'purple')

In [ ]:
# lag plot

from pandas.plotting import lag_plot

for i in [1,4,8,12,16,20,24,28,32,36,40]:
    lag_plot(df_cola['Sales'],lag = i)
    plt.show()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(df_cola['Sales'],label = 'Org')
for i in range(4,13,4):
    df_cola["Sales"].rolling(i).mean().plot(label=str(i))
plt.legend(loc='best')

In [ ]:
plt.figure(figsize=(14,4))
plt.plot(df_cola['Sales'])
plt.title("Original Series")
for i in range(2,7,2):
    plt.figure(figsize=(14,4))
    df_cola["Sales"].rolling(i).mean().plot(label=str(i))
    plt.title("Moving Average "+str(i))
    plt.legend(loc='best')
    plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decompose_ts_add = seasonal_decompose(df_cola.Sales,period=4)
with plt.rc_context():
    plt.rc("figure", figsize=(12,10))
    decompose_ts_add.plot()
    plt.show()

In [ ]:
import statsmodels.graphics.tsaplots as tsa_plots
with plt.rc_context():
    plt.rc("figure", figsize=(14,6))
    tsa_plots.plot_acf(df_cola.Sales,lags=40)
    plt.show()

In [ ]:
import statsmodels.graphics.tsaplots as tsa_plots
with plt.rc_context():
    plt.rc("figure", figsize=(14,6))
    tsa_plots.plot_pacf(df_cola.Sales,lags=20)
    plt.show()

In [ ]:
upsampled_month = cola1.drop(['Quarters','Q1','Q2','Q3','Q4'], axis=1)
upsampled_month = upsampled_month.resample('M').mean()
upsampled_month.head()

In [ ]:
interpolated_month = upsampled_month.interpolate(method='linear')
interpolated_month.head()

In [ ]:
interpolated_month.plot(figsize=(12,4))
plt.show()

In [ ]:
interpolated_month.reset_index(inplace=True)
interpolated_month['t'] = np.arange(1,len(interpolated_month)+1)
interpolated_month['t_squared'] = np.square(interpolated_month.t)
interpolated_month["month"] = interpolated_month.Date.dt.strftime("%b") # month extraction
interpolated_month["year"] = interpolated_month.Date.dt.strftime("%Y") # year extraction
months = pd.get_dummies(interpolated_month['month'])
months = months[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']]
df = pd.concat([interpolated_month,months],axis=1)
df['log_sales'] = np.log(df['Sales'])
df

In [ ]:
train = df.head(100)
test = df.tail(24)

In [ ]:
#Linear Model
import statsmodels.formula.api as smf

linear_model = smf.ols('Sales~t',data=train).fit()
pred_linear =  pd.Series(linear_model.predict(pd.DataFrame(test['t'])))
rmse_linear = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_linear))**2))
rmse_linear

In [ ]:
#Exponential Model

Exp = smf.ols('log_sales~t',data=train).fit()
pred_Exp = pd.Series(Exp.predict(pd.DataFrame(test['t'])))
rmse_Exp = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_Exp)))**2))
rmse_Exp

In [ ]:
#Quadratic

Quad = smf.ols('Sales~t+t_squared',data=train).fit()
pred_Quad = pd.Series(Quad.predict(test[["t","t_squared"]]))
rmse_Quad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_Quad))**2))
rmse_Quad

In [ ]:

#Additive seasonality

add_sea = smf.ols('Sales~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=train).fit()
pred_add_sea = pd.Series(add_sea.predict(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']]))
rmse_add_sea = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_add_sea))**2))
rmse_add_sea

In [ ]:
#Additive Seasonality Quadratic

add_sea_Quad = smf.ols('Sales~t+t_squared+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=train).fit()
pred_add_sea_quad = pd.Series(add_sea_Quad.predict(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','t','t_squared']]))
rmse_add_sea_quad = np.sqrt(np.mean((np.array(test['Sales'])-np.array(pred_add_sea_quad))**2))
rmse_add_sea_quad

In [ ]:
#Multiplicative Seasonality

Mul_sea = smf.ols('log_sales~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = train).fit()
pred_Mult_sea = pd.Series(Mul_sea.predict(test))
rmse_Mult_sea = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_Mult_sea)))**2))
rmse_Mult_sea

In [ ]:
#Multiplicative Additive Seasonality

Mul_Add_sea = smf.ols('log_sales~t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = train).fit()
pred_Mult_add_sea = pd.Series(Mul_Add_sea.predict(test))
rmse_Mult_add_sea = np.sqrt(np.mean((np.array(test['Sales'])-np.array(np.exp(pred_Mult_add_sea)))**2))
rmse_Mult_add_sea

In [ ]:
#Compare the results

data = {"MODEL":pd.Series(["rmse_linear","rmse_Exp","rmse_Quad","rmse_add_sea","rmse_add_sea_quad","rmse_Mult_sea","rmse_Mult_add_sea"]),"RMSE_Values":pd.Series([rmse_linear,rmse_Exp,rmse_Quad,rmse_add_sea,rmse_add_sea_quad,rmse_Mult_sea,rmse_Mult_add_sea])}
table_rmse=pd.DataFrame(data)
table_rmse.sort_values(['RMSE_Values'])

In [ ]:

# Since the additive seasonality quadratic model had the lowest rmse value, we choose this model for prediction.

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(train['Sales'], label='Train')
plt.plot(test['Sales'], label='Test')
plt.plot(pred_add_sea_quad, label='OLS Forecast')
plt.legend(loc='best')
plt.show()